In [1]:
import torch
import numpy as np
import os
import shutil
from tqdm import tqdm
import yaml

import sys
import os
curr_dir = os.getcwd()
if 's94zalek_hpc' in curr_dir:
    user_name = 's94zalek_hpc'
else:
    user_name = 's94zalek'
sys.path.append(f'/home/{user_name}/shape_matching')

# models
from my_code.models.diag_conditional import DiagConditionedUnet
from diffusers import DDPMScheduler

# training / evaluation
from torch.utils.tensorboard import SummaryWriter
from my_code.diffusion_training_sign_corr.train_model import train_epoch
from my_code.diffusion_training_sign_corr.validate_model import validate_epoch

import my_code.diffusion_training_sign_corr.data_loading as data_loading

from my_code.datasets.surreal_cached_train_dataset import SurrealTrainDataset
import networks.diffusion_network as diffusion_network  

In [6]:
import accelerate

In [7]:
experiment_name = 'pair_10_xy_distributed'
checkpoint_name = 'epoch_2'
subset_fraction = 100
# dataset_name = 'FAUST_orig'
dataset_name = 'SHREC19'


### config
exp_base_folder = f'/home/{user_name}/shape_matching/my_code/experiments/ddpm/{experiment_name}'
with open(f'{exp_base_folder}/config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)


### model
model = DiagConditionedUnet(config["model_params"]).to('cuda')
# model.load_state_dict(torch.load(f"{exp_base_folder}/checkpoints/{checkpoint_name}/model.safetensors"))
# model = model.to('cuda')

accelerate.load_checkpoint_in_model(model, f"{exp_base_folder}/checkpoints/{checkpoint_name}/model.safetensors")

In [13]:
#get the size of this file /home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/pair_10_xy_distributed/checkpoints/epoch_8/model.safetensors in GB
# os.path.getsize(f"{exp_base_folder}/checkpoints/{checkpoint_name}/model.safetensors") / 1024 / 1024 
# /home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/single_template_remeshed_augShapes_signNet_remeshed_mass_6b_1ev_10_0.2_0.8/checkpoints/checkpoint_99.pt
os.path.getsize('/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/single_template_remeshed_augShapes_signNet_remeshed_mass_6b_1ev_10_0.2_0.8/checkpoints/checkpoint_99.pt') / 1024 / 1024